<h2 align='center'>Codebasics ML Course: ML Flow Dagshub Tutorial</h2>

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='learnpythonlanguage', repo_name='mlflow_dagshub_demo', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6c6226bf-4e12-4933-99a2-333a71158a3b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c454c3c530b9f14f65aadbd9f20b429f43d84a1b0c5cac30698734bc84964990




Accessing as Codered-Vigneshvar

Initialized MLflow to track repo "learnpythonlanguage/mlflow_dagshub_demo"

Repository learnpythonlanguage/mlflow_dagshub_demo initialized!

In [10]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/03/31 13:14:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/904df44546484945bab3292236eb9148
🧪 View experiment at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/03/31 13:14:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/a17f39f3243c41c3b8e61606774bd6d5
🧪 View experiment at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/03/31 13:14:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/63f3014c029d4699b2940e7fdb908bc9
🧪 View experiment at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0


2025/03/31 13:14:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0/runs/ffa0bdd2286a49b8a54a711ae7657107
🧪 View experiment at: https://dagshub.com/Codered-Vigneshvar/mlflow_dagshub_demo.mlflow/#/experiments/0
